In [1]:
import os

import torch
import matplotlib.pyplot as plt
from torchvision import transforms

import numpy as np 
import cv2 
from PIL import Image
from copan_border.model.segmentation import SegmentationModels

/home/ffrangiamore/anaconda3/envs/copan_newlab_border-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = SegmentationModels("FPN", "resnet34", in_channels=3, out_classes=1)

In [3]:
# v11 best for back - external
# v14 best for back - internal

In [4]:
model = SegmentationModels.load_from_checkpoint('C:\\Progetti\\Copan\\copan_border\\notebook\\training\\checkpoints\\best_model-v17.ckpt', arch="fpn", encoder_name="resnet34", in_channels=3, out_classes=1)

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/c/Progetti/Copan/copan_border/notebook/training/C:\\Progetti\\Copan\\copan_border\\notebook\\training\\checkpoints\\best_model-v17.ckpt'

In [ ]:
model.eval()

In [ ]:
import time

In [ ]:
input_image = "C:\\Progetti\\Copan\\copan_border\\halcon\\external\\test"
for img in os.listdir(input_image):
    if img.endswith(".png"):
        image_path = os.path.join("C:\\Progetti\\Copan\\copan_border\\halcon\\external\\test", img)
        transform = transforms.Compose([
            transforms.ToTensor(),  # Convert image to tensor
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize if used during training
            transforms.Resize((512, 512))
        ])

        # Step 3: Load and preprocess the image
        
        image = Image.open(image_path).convert("RGB")  # Open image and ensure it's in RGB format
        start = time.time()
        # Apply the same transformations used during training
        input_image = transform(image).unsqueeze(0)  # Add a batch dimension (N, C, H, W)

        # Step 4: Run inference (pass the image through the model)
        with torch.no_grad():  # No need to track gradients during inference
            output = model(input_image.cuda())
        output = output.sigmoid()
        
        output_binary = output[0][0].cpu().numpy()
        
        output_binary[output_binary>0.6]=1
        output_binary[output_binary<=0.6]=0
        output_binary = output_binary.astype(np.uint8)
        output_binary = cv2.resize(output_binary, (601, 603))
         
        
        end = time.time()
        print(end - start)

        contours, _ = cv2.findContours(output_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Create a copy of the original image to overlay the border
        overlay_image = np.array(image).copy()
        

        # Draw the contour border on the overlay image
        overlapped = cv2.drawContours(overlay_image, contours, -1, (255, 0, 0), 2)  # Green border with thickness 2
        cv2.imwrite(os.path.join("C:\\Progetti\\Copan\\copan_border\\immagini\\cropped_image\\external\\pred_test", img), cv2.cvtColor(overlapped, cv2.COLOR_RGB2BGR))


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Progetti\\Copan\\copan_border\\halcon\\external\\test\\'